In [ ]:
!pip install openai
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
api_key_ijas = ""
api_key_eyza = ""

import re
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from time import sleep
import numpy as np
root =  '/content/drive/MyDrive/ChatGPT'

Mounted at /content/drive


In [ ]:
openai.organization = "" # KAIST(VOICES) organization
openai.api_key = api_key_ijas # janlup ganti

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ChatGPT/IndoCollex.csv')
for column in df.columns:
    if column == "no":
        continue
    else:
        df[column] = df[column].astype(pd.StringDtype()) 

df

,no,transformed,original-for,transformation
0,19,sampe,sampai,sound-alter
1,29,dgn,dengan,disemvoweling
2,32,org,orang,disemvoweling
3,34,karna,karena,disemvoweling
4,35,tu,itu,disemvoweling
...,...,...,...,...
2120,947,inii,ini,sound-alter
2121,1266,alhamdulillaah,alhamdulillah,sound-alter
2122,x,bls,balas,disemvoweling
2123,37,disini,di sini,space-dash


In [ ]:
df.value_counts(["transformation"])

transformation
sound-alter       626
disemvoweling     418
affixation        332
space-dash        288
shorten           228
acronym           190
rev                43
dtype: int64

## Util

In [ ]:
def infer(order, questions, iteration, try_read = False, path=None, temp=0.0):
    try:
      if try_read:
        collect = pd.read_csv(path)
      else:
        collect = pd.DataFrame()
    except FileNotFoundError:
      collect = pd.DataFrame()
    
    question_len = len(questions)
    while len(collect.columns) < iteration:
      now = list()

      for i, q in enumerate(questions):
        tries = 0
        while(1):
          try:
            response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                                    temperature=temp,
                                                    messages=[{"role": "system", "content": "Anda adalah ChatGPT."},
                                                              {"role": "user", "content": f"{order}.\n {q}."}])
            
            if i % (question_len//10) == 0:
              print(f'Question {i+1}/{question_len} Completed')

            now.append(response['choices'][0]['message']['content'])
            #print(response['choices'][0]['message']['content'])
            break

          except:
            if tries > 3:
              print('More than 3 tries, terminating')
              break

            tries += 1
            print(f'RateLimitError {tries}, waiting')
            sleep(60*tries)
            
        if tries > 3:
          break
      if tries > 3:
        break

      print(f'Iteration {len(collect.columns)+1} Completed')
      collect[len(collect.columns)+1] = now
      sleep(120)

    return collect

In [ ]:
def clean(x):
  return ''.join(re.findall(r'(?:\w* ?)+', x)).lower()

In [ ]:
def accuracy(inf, ans):
  correct = 0
  for a, b in zip(inf, ans):
    if b.lower() in a.lower(): correct+=1
  return correct/len(inf)

def eval(res, df):
  res['clean'] = res['1'].apply(clean)
  print('overall:', accuracy(res['clean'], df['original-for']))
  for trans in desc.keys():
    inf = res.loc[df['transformation'] == trans]
    ans = df.loc[df['transformation'] == trans]
    print(f'{trans}:', accuracy(inf['clean'], ans['original-for']))

In [ ]:
desc = {
    "disemvoweling":"menghapus beberapa atau semua vokal",
    "affixation":"memodifikasi, menambah, atau menghilangkan afiks",
    "shorten":"memendekkan dari kata aslinya",
    "space-dash":"menghapus spasi/tanda hubung",
    "sound-alter":"sedikit merubah baik suara dan/atau ejaan dalam teks",
    "acronym":"menggabungkan huruf atau suku kata dari satu atau lebih kata yang mirip dengan akronim, singkatan, dan portmanteau",
    "rev":"membalikkan huruf, atau biasa dikenal sebagai 'Boso Walikan'"
}

In [ ]:
examples = {
    "disemvoweling":"kemarin -> kmrn, belum -> blum, besok -> bsk",
    "affixation":"menyanyikan -> nyanyiin, mengabari -> ngabarin",
    "shorten":"sudah -> dah, internet -> inet, halusinasi -> halu",
    "space-dash":"di rumah -> dirumah, terima kasih -> terimakasih, ibu-ibu -> ibu2",
    "sound-alter":"pakai -> pake, pahit -> pait, aku -> akuh",
    "acronym":"ibu hamil -> bumil, budak cinta -> bucin, anak baru gede -> abg",
    "rev":"malang -> ngalam, bang -> ngab"
}

## Informal to Formal

In [ ]:
infer_IF = True
if infer_IF:
    order = "Buatlah bentuk formal dari sebuah kata berikut. Jawablah hanya dengan kata formal yang dimaksud dan jangan berikan penjelasan dari jawaban anda. Jawab dengan format 'Kata Formal: <jawaban>'"
    words = df['transformed'].values

    questions = []

    for i in range(len(words)):
        questions.append(f"Kata anda adalah '{words[i]}'")

    print(questions[1])

    res = infer(order=order, 
                questions=questions, 
                iteration=1)

    print(res)
    res.to_csv('/content/drive/MyDrive/ChatGPT/formal_inferred_3.csv')

Kata anda adalah 'dgn'
Question 1/2125 Completed
Question 213/2125 Completed
Question 425/2125 Completed
Question 637/2125 Completed
Question 849/2125 Completed
Question 1061/2125 Completed
Question 1273/2125 Completed
Question 1485/2125 Completed
Question 1697/2125 Completed
Question 1909/2125 Completed
Question 2121/2125 Completed
Iteration 1 Completed
                                                      1
0                                  Kata Formal: Sampai.
1                                  Kata Formal: Dengan.
2                              Kata Formal: Organisasi.
3                                  Kata Formal: Karena.
4                                     Kata Formal: itu.
...                                                 ...
2120  Kata Formal: Tidak ada. 'inii' bukanlah kata f...
2121                        Kata Formal: Alhamdulillah.
2122                                Kata Formal: Balas.
2123                              Kata Formal: Di sini.
2124  Kata Formal: Tidak ad

### Accuracy Check

In [ ]:
res = pd.read_csv('/content/drive/MyDrive/ChatGPT/formal_inferred_3.csv')
res

,Unnamed: 0,1
0,0,Kata Formal: Sampai.
1,1,Kata Formal: Dengan.
2,2,Kata Formal: Organisasi.
3,3,Kata Formal: Karena.
4,4,Kata Formal: itu.
...,...,...
2120,2120,Kata Formal: Tidak ada. 'inii' bukanlah kata f...
2121,2121,Kata Formal: Alhamdulillah.
2122,2122,Kata Formal: Balas.
2123,2123,Kata Formal: Di sini.


In [ ]:
eval(res, df)

overall: 0.3976470588235294
disemvoweling: 0.5263157894736842
affixation: 0.5451807228915663
shorten: 0.25877192982456143
space-dash: 0.18055555555555555
sound-alter: 0.41054313099041534
acronym: 0.39473684210526316
rev: 0.023255813953488372


## Formal to Informal

### Zero-Shot

In [ ]:
infer_FI_0 = True
if infer_FI_0:   
    order = "Buatlah bentuk informal yang sering dipakai dalam percakapan sehari-hari dari sebuah kata berikut. Jawablah hanya dengan kata informal yang dimaksud dan jangan berikan penjelasan dari jawaban anda. Jawab dengan format 'Kata Informal: <jawaban>'"

    words = df['original-for'].values
    methods = df['transformation'].values
    questions = []

    for i in range(len(words)):
        questions.append(f"Ubah kata yang diberikan dengan {desc[methods[i]]}.\nKata anda adalah '{words[i]}'")

    print(questions[1])

    res = infer(order=order, 
                questions=questions, 
                iteration=1)

    res
    res.to_csv('/content/drive/MyDrive/ChatGPT/zero_shot_informal_inferred_3.csv')

Ubah kata yang diberikan dengan menghapus beberapa atau semua vokal.
Kata anda adalah 'dengan'
Question 1/2125 Completed
Question 213/2125 Completed
Question 425/2125 Completed
Question 637/2125 Completed
Question 849/2125 Completed
Question 1061/2125 Completed
Question 1273/2125 Completed
Question 1485/2125 Completed
Question 1697/2125 Completed
Question 1909/2125 Completed
Question 2121/2125 Completed
Iteration 1 Completed


In [ ]:
res = pd.read_csv('/content/drive/MyDrive/ChatGPT/zero_shot_informal_inferred_3.csv')
res

,Unnamed: 0,1
0,0,Kata Informal: sampe
1,1,Kata Informal: dgn
2,2,Kata Informal: org
3,3,Kata Informal: krn
4,4,Kata Informal: t.
...,...,...
2120,2120,Kata Informal: nih
2121,2121,Kata Informal: Alhamdulah
2122,2122,Kata Informal: bls
2123,2123,Kata Informal: disini


In [ ]:
eval(res, df)

overall: 0.10164705882352941
disemvoweling: 0.01674641148325359
affixation: 0.060240963855421686
shorten: 0.14473684210526316
space-dash: 0.024305555555555556
sound-alter: 0.22364217252396165
acronym: 0.03684210526315789
rev: 0.046511627906976744


### Few-**Shot**

In [ ]:
infer_FI_few = True
if infer_FI_few:   
    order = "Buatlah bentuk informal yang sering dipakai dalam percakapan sehari-hari dari sebuah kata berikut. Jawablah hanya dengan kata informal yang dimaksud dan jangan berikan penjelasan dari jawaban anda. Jawab dengan format 'Kata Informal: <jawaban>'"

    words = df['original-for'].values
    methods = df['transformation'].values
    questions = []

    for i in range(len(words)):
        questions.append(f"Ubah kata yang diberikan dengan {desc[methods[i]]}. Contohnya adalah {examples[methods[i]]}.\nKata anda adalah '{words[i]}'")

    print(questions[1])

    res = infer(order=order, 
                questions=questions, 
                iteration=1)

    res
    res.to_csv('/content/drive/MyDrive/ChatGPT/few_shot_informal_inferred_3.csv')

Ubah kata yang diberikan dengan menghapus beberapa atau semua vokal. Contohnya adalah kemarin -> kmrn, belum -> blum, besok -> bsk.
Kata anda adalah 'dengan'
Question 1/2125 Completed
RateLimitError 1, waiting
Question 213/2125 Completed
Question 425/2125 Completed
Question 637/2125 Completed
Question 849/2125 Completed
Question 1061/2125 Completed
Question 1273/2125 Completed
Question 1485/2125 Completed
Question 1697/2125 Completed
Question 1909/2125 Completed
Question 2121/2125 Completed
Iteration 1 Completed


In [ ]:
res = pd.read_csv('/content/drive/MyDrive/ChatGPT/few_shot_informal_inferred_3.csv')
res

,Unnamed: 0,1
0,0,Kata Informal: sampe
1,1,Kata Informal: dgn
2,2,Kata Informal: org
3,3,Kata Informal: krn
4,4,Kata Informal: tuh
...,...,...
2120,2120,Kata Informal: nih
2121,2121,Kata Informal: Alhamdulah
2122,2122,Kata Informal: bls
2123,2123,Kata Informal: disini


In [ ]:
eval(res, df)

overall: 0.1312941176470588
disemvoweling: 0.06698564593301436
affixation: 0.09337349397590361
shorten: 0.11403508771929824
space-dash: 0.017361111111111112
sound-alter: 0.2715654952076677
acronym: 0.010526315789473684
rev: 0.3953488372093023


#Results

In [ ]:
res = pd.read_csv(f"/content/drive/MyDrive/ChatGPT/few_shot_informal_inferred_3.csv")
eval(res, df)

overall: 0.1312941176470588
disemvoweling: 0.06698564593301436
affixation: 0.09337349397590361
shorten: 0.11403508771929824
space-dash: 0.017361111111111112
sound-alter: 0.2715654952076677
acronym: 0.010526315789473684
rev: 0.3953488372093023
